In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
!gdown 1izxctyA4b0anBkDgTrpppirwEXgSAwEY

filepath = '/content/web_data.csv'
df = pd.read_csv(filepath)
df.drop_duplicates(keep='first', inplace=True)
df.drop("Location", axis = 1, inplace = True)
print(df.shape)
df.head()

Downloading...
From: https://drive.google.com/uc?id=1izxctyA4b0anBkDgTrpppirwEXgSAwEY
To: /content/web_data.csv
100% 2.36M/2.36M [00:00<00:00, 168MB/s]
(10229, 2)


,Category,Incident
0,Indecent exposure,I'm a 58 year old woman and not used to seein...
1,Touching /Groping,a boy with whom I was in relationship with ca...
2,Stalking,I was stalked in this street and cat-called. ...
3,Touching /Groping,groped in the middle of the road \n\t\t\t\t\...
4,Stalking,I go to the race course for early morning run...


In [4]:
df['Category'] = df['Category'].astype("category")

In [5]:
num_of_labels = len(df['Category'].unique())

In [6]:
X = df['Incident']
y = df['Category']

In [7]:
X = [row.replace(" \n\t\t\t\t\t\t  More Information »\n« Less Information\n", '') for row in X]

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [10]:
# X_train = X_train.tolist()
y_train = y_train.tolist()
# X_test = X_test.tolist()
y_test = y_test.tolist()

In [11]:
# from transformers import DistilBertTokenizerFast

# tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
# inputs = tokenizer(X_train, padding="max_length", truncation=True)

In [12]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
inputs = tokenizer(X_train, padding="max_length", truncation=True)

In [13]:
import torch

class feedbackDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = feedbackDataset(inputs, y_train)

In [14]:
print(train_dataset.__getitem__(2))

{'input_ids': tensor([    0,    83,   733,    76,   793,  2943,    21, 33298,   196,   259,
            4,  1437,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1, 

In [15]:
# from transformers import DistilBertForSequenceClassification

# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_of_labels)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [16]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = num_of_labels)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
            output_dir = './results', 
            num_train_epochs = 3,
            per_device_train_batch_size = 16,
            per_device_eval_batch_size = 64, 
            warmup_steps = 500,
            weight_decay = 0.01,
            logging_dir = './logs', 
            logging_steps = 10
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def to_check_result(test_encoding):

    input_ids = torch.tensor(test_encoding['input_ids']).to(device)
    attention_mask = torch.tensor(test_encoding['attention_mask']).to(device)

    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    
    y = np.argmax(outputs[0].to('cpu').numpy())

    return y

In [ ]:
 l2 = []

for i in X_test:

    test_encoding1 = tokenizer(i, truncation=True, padding=True)
    # input_ids = torch.tensor(test_encoding1['input_ids']).to(device)
    # attention_mask = torch.tensor(test_encoding1['attention_mask']).to(device)
    op = to_check_result(test_encoding1)
    l2.append(op)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, l2)
acc

**DistilBERT** - 0.5762 <br>
**RoBERTa** - 0.5824<br>
**DeBERTa** - CUDA out of memory<br>